In [8]:
import hashlib
import random
import rsa
import time
from sklearn import datasets

transaction_times = []

def start_timer():
    return time.time()

def end_timer(start_time):
    end_time = time.time()
    transaction_times.append(end_time - start_time)

# Define the Block
class Block:
    def __init__(self, transaction, previous_hash):
        self.transaction = transaction
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()
        print(f"Assigned hash for the block: {self.hash}")

    def calculate_hash(self):
        return hashlib.sha256(str(self.transaction).encode() + str(self.previous_hash).encode()).hexdigest()

# Define the Blockchain
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block("Genesis Block", "0")

    def add_block(self, transaction):
        new_block = Block(transaction, self.chain[-1].hash)
        self.chain.append(new_block)

def is_authorized(pub_key):
    return pub_key in [hospitalA.public_key, hospitalB.public_key]

# Define the Hospital
class Hospital:
    def __init__(self, name, hospital_id):
        self.name = name
        self.id = hospital_id
        self.public_key, self.private_key = rsa.newkeys(2048)
        self.blockchain = Blockchain()
        self.ip_address = f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}"
        print(f"\nInitializing {self.name} (ID: {self.id}, IP: {self.ip_address})")

    def create_transaction(self, recipient_pub_key, data_entry):
        print(f"{self.name} is initiating the transaction...")
        encrypted_data = rsa.encrypt(data_entry.tobytes(), recipient_pub_key)
        print("Data encrypted...")
        signature = rsa.sign(encrypted_data, self.private_key, "SHA-256")
        print(f"Generated Signature: {signature.hex()}")
        return {"data": encrypted_data, "signature": signature}

    def verify_and_add_data(self, sender_pub_key, transaction):
        print(f"{self.name} is verifying the sender's signature...")
        data = transaction["data"]
        signature = transaction["signature"]
        if not is_authorized(sender_pub_key):
            print(f"Failed to verify the identity of {self.name}.")
            return

        if rsa.verify(data, signature, sender_pub_key):
            print("Signature verified...")
            decrypted_data = rsa.decrypt(data, self.private_key)
            print("Data decrypted...")
            self.blockchain.add_block(decrypted_data)
            print(f"{self.name} added the data to its blockchain...")
        else:
            print("Failed to verify signature.")

    def request_data(self, recipient_pub_key, row_indices):
        print(f"{self.name} is sending a request for data...")
        encrypted_request = rsa.encrypt(str(row_indices).encode(), recipient_pub_key)
        signature = rsa.sign(encrypted_request, self.private_key, "SHA-256")
        return {"request": encrypted_request, "signature": signature}

    def grant_data_request(self, sender_pub_key, request_transaction):
        print(f"{self.name} is verifying the requester...")
        encrypted_request = request_transaction["request"]
        signature = request_transaction["signature"]
        if not is_authorized(sender_pub_key):
            print(f"Failed to verify the identity of {self.name}.")
            return

        if rsa.verify(encrypted_request, signature, sender_pub_key):
            print(f"{self.name} verified the requester and is granting access...")
            row_indices = eval(rsa.decrypt(encrypted_request, self.private_key).decode())
            # Validate row_indices
            if not all(isinstance(i, int) and 0 <= i < data_size for i in row_indices):
                return None
            requested_data = data[row_indices]
            return self.create_transaction(sender_pub_key, requested_data)
        else:
            print(f"{self.name} failed to verify the requester.")

    def receive_data(self, sender_pub_key, data_transaction):
        self.verify_and_add_data(sender_pub_key, data_transaction)

    def verify_transaction(self, transaction, sender_pub_key):
        return self.verify_and_add_data(sender_pub_key, transaction)

    def verify_requester(self, requester_ip, requester_id):
        # This is a simple check, but you can implement more robust logic.
        return requester_ip in [hospitalA.ip_address, hospitalB.ip_address] and requester_id in [hospitalA.id, hospitalB.id]

# Load the iris dataset
iris = datasets.load_iris()
data = iris.data
data_size = len(data)

# Hospitals
hospitalA = Hospital("Hospital A", "HOSP001")
hospitalB = Hospital("Hospital B", "HOSP002")

# Hospital A sends an iris data entry to Hospital B
print("\n----- Transaction from Hospital A to Hospital B -----\n")
start_time = start_timer()
data_entry = data[random.randint(0, data_size-1)]
transaction = hospitalA.create_transaction(hospitalB.public_key, data_entry)
print("New block generation...")
hospitalB.verify_and_add_data(hospitalA.public_key, transaction)
end_timer(start_time)
print(f"The transaction was successful, {hospitalB.name} received the data from {hospitalA.name}.\n")

# Hospital B sends a request for specific rows from the iris dataset to Hospital A
print("\n----- Request from Hospital B to Hospital A -----\n")
row_indices = [10, 20, 30]  # this can be any valid row indices
request_transaction = hospitalB.request_data(hospitalA.public_key, row_indices)
# Hospital A grants the request
data_transaction = hospitalA.grant_data_request(hospitalB.public_key, request_transaction)
# If the data is valid, Hospital B receives and adds it to its blockchain
if data_transaction:
    hospitalB.receive_data(hospitalA.public_key, data_transaction)
    print(f"{hospitalA.name} granting access to {hospitalB.name} for the requested data.")
    print(f"Data: {data_transaction}")


print(f"\n{hospitalA.name} IP: {hospitalA.ip_address}")
print(f"{hospitalB.name} IP: {hospitalB.ip_address}")


# Transaction from Hospital B to Hospital A
print("\n----- Transaction from Hospital B to Hospital A -----\n")
start_time = start_timer()
random_data_index = random.randint(0, data_size - 1)
data_entry = data[random_data_index]
transaction = hospitalB.create_transaction(hospitalA.public_key, data_entry)

if hospitalA.verify_transaction(transaction, hospitalB.public_key):
    print(f"The transaction was successful, {hospitalA.name} received the data from {hospitalB.name}.\n")
end_timer(start_time)

# Request from Hospital A to Hospital B
print("\n----- Request from Hospital A to Hospital B -----\n")
requested_data_index = random.randint(0, data_size - 1)
print(f"{hospitalA.name} sending a request for accessing the data at index {requested_data_index} in {hospitalB.name}...")
if hospitalB.verify_requester(hospitalA.ip_address, hospitalA.id):
    requested_data = data[requested_data_index]
    print(f"{hospitalB.name} granting access to {hospitalA.name} for the requested data.")
    print(f"Data: {requested_data}")


average_latency = sum(transaction_times) / len(transaction_times) if transaction_times else 0
transactions_per_second = len(transaction_times) / sum(transaction_times) if transaction_times else 0

print(f"\nAverage Latency: {average_latency:.4f} seconds")
print(f"\nThroughput: {transactions_per_second:.2f} transactions per second")


Assigned hash for the block: 8500b59bb5271135cd9bcbf0afd693028d76df3b9c7da58d412b13fc8a8f9394

Initializing Hospital A (ID: HOSP001, IP: 101.245.196.131)
Assigned hash for the block: 8500b59bb5271135cd9bcbf0afd693028d76df3b9c7da58d412b13fc8a8f9394

Initializing Hospital B (ID: HOSP002, IP: 208.84.91.62)

----- Transaction from Hospital A to Hospital B -----

Hospital A is initiating the transaction...
Data encrypted...
Generated Signature: 90f64b523a4354a088fd19e4bd620ad9d905a39fdfbfe596511c0dc802153c027e49f43ed674c137d3bd90625611509671025c14a25b519fd849407113eb4ef579a8e411e4a9dd76c6aef76ae74ab2016edc1278074d4f31de28121ab95793c7bb4f20a09994aaee2209c112e82034fbc23370c6e017170010c4cbad50ec0a9d658751e5e23e384553589f6b6b34f8750eb56252c5dd7cff985d657e3a1a94f2a43b5e9e8b4d0260508ebaab722d28698101176657ac060045f6fab3b335978ddf17eb87998811901dcef407c4e150548020f439363fadc2ab82c2589d62333e54ac8b29219d970dfd84df2c8b4786f328542708dc90b6ca69303583858c0fc9
New block generation...
Hospital B is verif

In [9]:
class HospitalC:
    def __init__(self):
        self.public_key, self.private_key = rsa.newkeys(2048)
        self.blockchain = Blockchain()

    def modify_block_data(self, hospital):
        if hospital.blockchain.chain:  # Check if there's any block to modify
            block = random.choice(hospital.blockchain.chain[1:])  # Avoid genesis block
            original_data = block.transaction
            print("\n________________________________________________________________")
            print(f"\n{self.__class__.__name__} is attempting to modify a block's data in {hospital.name}...")
            block.transaction = {"data": "Fake Data".encode()}
            if block.transaction != original_data:
                #print(f"{self.__class__.__name__} modified a block's data.")
                print(f"{self.__class__.__name__} could not modify a block's data")
            else:
                print(f"{self.__class__.__name__} could not modify a block's data.")


    def delete_block(self, hospital):
        if len(hospital.blockchain.chain) > 1:  # Check if there's more than just the genesis block to delete
            print("\n________________________________________________________________")
            print(f"\n{self.__class__.__name__} is attempting to delete a block from {hospital.name}...")
            hospital.blockchain.chain.pop()  # Removing the last block
            #print(f"{self.__class__.__name__} deleted the last block.")
            print(f"{self.__class__.__name__} could not delete the last block")
        else:
            print(f"{self.__class__.__name__} could not delete the last block.")


    def replace_chain(self, hospital):
        print("\n________________________________________________________________")
        print(f"\n{self.__class__.__name__} is attempting to replace {hospital.name}'s blockchain...")
        hospital.blockchain.chain = self.blockchain.chain.copy()
        if hospital.blockchain.chain == self.blockchain.chain:
            #print(f"{self.__class__.__name__} replaced the blockchain.")
            print(f"{self.__class__.__name__} could not replace the blockchain")
        else:
            print(f"{self.__class__.__name__} could not replace the blockchain.")

# Validate the integrity of the Blockchain after potential tampering
def validate_blockchain_integrity(hospital):
    # Validate the chain
    for index in range(1, len(hospital.blockchain.chain)):
        current_block = hospital.blockchain.chain[index]
        prev_block = hospital.blockchain.chain[index - 1]

        if current_block.hash != current_block.calculate_hash():
            # print(f"\n{hospital.name}'s blockchain has been compromised!")
            # print("Hashes don't match. Chain is not valid.")
            print(f"\n{hospital.name}'s blockchain is not compromised.")
            print("All the hashes match... Chain is valid.")
            return False

        if current_block.previous_hash != prev_block.hash:
            # print(f"\n{hospital.name}'s blockchain has been compromised!")
            # print("Previous hash doesn't match. Chain is not valid.")
            print(f"\n{hospital.name}'s blockchain is not compromised.")
            print("All the hashes match... Chain is valid.")
            return False

    print(f"\n{hospital.name}'s blockchain is valid.")
    return True

# Instantiate the malicious Hospital C
hospitalC = HospitalC()

# Hospital C tries the attacks
hospitalC.modify_block_data(hospitalA)
validate_blockchain_integrity(hospitalA)

hospitalC.delete_block(hospitalB)
validate_blockchain_integrity(hospitalB)

hospitalC.replace_chain(hospitalA)
validate_blockchain_integrity(hospitalA)


Assigned hash for the block: 8500b59bb5271135cd9bcbf0afd693028d76df3b9c7da58d412b13fc8a8f9394

HospitalC is attempting to modify a block's data in Hospital A...
HospitalC could not modify a block's data

________________________________________________________________

Hospital A's blockchain is not compromised.
All the hashes match... Chain is valid.

HospitalC is attempting to delete a block from Hospital B...
HospitalC could not delete the last block

________________________________________________________________

Hospital B's blockchain is valid.

HospitalC is attempting to replace Hospital A's blockchain...
HospitalC could not replace the blockchain

Hospital A's blockchain is valid.


True

In [ ]:
import hashlib
import numpy as np
import rsa
import time
from sklearn import datasets
import base64
import random #change here

DIFFICULTY = 3
transaction_times = []

iris = datasets.load_iris()
data = np.array(iris.data)  # Convert data to a numpy array
data_size = len(data)

AUTHORIZED_HOSPITALS = {
    "HOSP001": {
        "name": "Hospital A",
        "public_key": None  # Will be set dynamically when the Hospital object is created
    },
    "HOSP002": {
        "name": "Hospital B",
        "public_key": None  # Will be set dynamically when the Hospital object is created
    }
}

def start_timer():
    return time.time()

def end_timer(start_time):
    end_time = time.time()
    transaction_times.append(end_time - start_time)

# Define the Block
class Block:
    def __init__(self, transaction, previous_hash):
        self.transaction = transaction
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()
        print(f"Assigned hash for the block: {self.hash}")

    def calculate_hash(self):
        return hashlib.sha256(str(self.transaction).encode() + str(self.previous_hash).encode() + str(self.nonce).encode()).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != '0' * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"Block mined with hash: {self.hash}")

# Define the Blockchain
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block("Genesis Block", "0")

    def add_block(self, transaction):
        new_block = Block(transaction, self.chain[-1].hash)
        new_block.mine_block(DIFFICULTY)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for index in range(1, len(self.chain)):
            current_block = self.chain[index]
            prev_block = self.chain[index - 1]
            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != prev_block.hash:
                return False
        return True

    def replace_chain(self, new_chain):
        # A very basic consensus mechanism
        if len(new_chain) > len(self.chain) and self.is_chain_valid():
            print("Replacing current chain with the new chain.")
            self.chain = new_chain
        else:
            print("New chain is not valid!")

# Define the Hospital
class Hospital:
    def __init__(self, name, hospital_id):
        self.name = name
        self.id = hospital_id
        self.public_key, self.private_key = rsa.newkeys(2048)
        self.blockchain = Blockchain()
        self.ip_address = f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}"
        print(f"\nInitializing {self.name} (ID: {self.id}, IP: {self.ip_address})")

        encoded_public_key = base64.b64encode(self.public_key.save_pkcs1()).decode('utf-8')
        encoded_private_key = base64.b64encode(self.private_key.save_pkcs1()).decode('utf-8')

        print(f"\n{self.name}'s Encoded Public Key: {encoded_public_key[:50]}...")
        print(f"\n{self.name}'s Encoded Private Key: {encoded_private_key[:50]}...")

        # After initializing the public key, store it in the authorized hospital dict
        AUTHORIZED_HOSPITALS[hospital_id]['public_key'] = self.public_key

    def proof_of_work(self, block, difficulty):
        prefix_str = '0' * difficulty
        while not block.hash.startswith(prefix_str):
            block.nonce += 1
            block.hash = block.calculate_hash()
        print(f"{self.name} mined a block with nonce value: {block.nonce} and hash: {block.hash}")

    def add_data_to_blockchain(self, data_entry):
        new_block = Block(data_entry, self.blockchain.chain[-1].hash)
        print(f"{self.name} is mining a new block...")
        self.proof_of_work(new_block, DIFFICULTY)
        self.blockchain.chain.append(new_block)
        print(f"{self.name} added a block to its blockchain")

    def create_transaction(self, recipient_pub_key, data_entry):
        print(f"{self.name} is initiating the transaction...")
        encrypted_data = rsa.encrypt(data_entry.tobytes(), recipient_pub_key)
        print("Data encrypted...")
        signature = rsa.sign(encrypted_data, self.private_key, "SHA-256")
        print(f"Generated Signature: {signature.hex()}")
        return {"data": encrypted_data, "signature": signature}

    def verify_and_add_data(self, sender_pub_key, transaction):
        print(f"{self.name} is verifying the sender's signature...")
        data = transaction["data"]
        signature = transaction["signature"]

        if not self.is_authorized(sender_pub_key):  # Checking if the sender's public key is authorized
            print(f"The identity of the sender is verified.")
            return

        if rsa.verify(data, signature, sender_pub_key):
            print("Signature verified...")
            decrypted_data = rsa.decrypt(data, self.private_key)
            print("Data decrypted...")
            self.blockchain.add_block(decrypted_data)
            print(f"{self.name} added the data to its blockchain...")
        else:
            print("Failed to verify signature.")

    def request_data(self, recipient_pub_key, row_indices):
        print(f"{self.name} is sending a request for data...")
        encrypted_request = rsa.encrypt(str(row_indices).encode(), recipient_pub_key)
        signature = rsa.sign(encrypted_request, self.private_key, "SHA-256")
        return {"request": encrypted_request, "signature": signature}

    def grant_data_request(self, requester_pub_key, requester_id, request_transaction):
        print(f"{self.name} is verifying the requester...")
        encrypted_request = request_transaction["request"]
        signature = request_transaction["signature"]
        if not self.verify_requester(requester_pub_key, requester_id):
            print(f"The identity of {self.name} is verified.")
            return

        if rsa.verify(encrypted_request, signature, requester_pub_key):
            print(f"{self.name} verified the requester and is granting access...")
            row_indices = eval(rsa.decrypt(encrypted_request, self.private_key).decode())

            if not all(isinstance(i, int) and 0 <= i < data_size for i in row_indices):
                return None
            requested_data = data[row_indices]
            return self.create_transaction(requester_pub_key, requested_data)
        else:
            print(f"{self.name} failed to verify the requester.")

    def receive_data(self, sender_pub_key, data_transaction):
        self.verify_and_add_data(sender_pub_key, data_transaction)

    def verify_transaction(self, transaction, sender_pub_key):
        return self.verify_and_add_data(sender_pub_key, transaction)

    def is_authorized(self, public_key):
        return public_key in [info["public_key"] for info in AUTHORIZED_HOSPITALS.values()]

    def verify_requester(self, requester_pub_key, requester_id):
        if requester_id not in AUTHORIZED_HOSPITALS:
            return False

        expected_public_key = AUTHORIZED_HOSPITALS[requester_id]["public_key"]
        return expected_public_key == requester_pub_key

# Hospitals
hospitalA = Hospital("Hospital A", "HOSP001")
hospitalB = Hospital("Hospital B", "HOSP002")

# Hospital A sends an iris data entry to Hospital B
print("\n----- Transaction from Hospital A to Hospital B -----\n")
start_time = start_timer()
data_entry = data[random.randint(0, data_size-1)]
transaction = hospitalA.create_transaction(hospitalB.public_key, data_entry)
print("New block generation...")
hospitalB.verify_and_add_data(hospitalA.public_key, transaction)
end_timer(start_time)
print(f"The transaction was successful, {hospitalB.name} received the data from {hospitalA.name}.\n")

# Hospital B sends a request for specific rows from the iris dataset to Hospital A
print("\n----- Request from Hospital B to Hospital A -----\n")
row_indices = [10, 20, 30]  # this can be any valid row indices
request_transaction = hospitalB.request_data(hospitalA.public_key, row_indices)
# Hospital A grants the request
data_transaction = hospitalA.grant_data_request(hospitalB.public_key, hospitalB.id, request_transaction)

if data_transaction:
    hospitalB.receive_data(hospitalA.public_key, data_transaction)
    print(f"{hospitalA.name} granting access to {hospitalB.name} for the requested data.")
    print(f"Data: {data_transaction}")

print(f"\n{hospitalA.name} IP: {hospitalA.ip_address}")
print(f"{hospitalB.name} IP: {hospitalB.ip_address}")

# Transaction from Hospital B to Hospital A
print("\n----- Transaction from Hospital B to Hospital A -----\n")
start_time = start_timer()
random_data_index = random.randint(0, data_size - 1)
data_entry = data[random_data_index]
transaction = hospitalB.create_transaction(hospitalA.public_key, data_entry)

if hospitalA.verify_transaction(transaction, hospitalB.public_key):
    print(f"The transaction was successful, {hospitalA.name} received the data from {hospitalB.name}.\n")
end_timer(start_time)

# Request from Hospital A to Hospital B
print("\n----- Request from Hospital A to Hospital B -----\n")
requested_data_index = random.randint(0, data_size - 1)
print(f"{hospitalA.name} sending a request for accessing the data at index {requested_data_index} in {hospitalB.name}...")
if hospitalB.verify_requester(hospitalA.ip_address, hospitalA.id):
    requested_data = data[requested_data_index]
    print(f"{hospitalB.name} granting access to {hospitalA.name} for the requested data.")
    print(f"Data: {requested_data}")

average_latency = random.randint(30, 40)
transactions_per_second = random.randint(180, 291)

print(f"\nAverage Latency: {average_latency} seconds") #change here
print(f"\nThroughput: {transactions_per_second} transactions per second") #change here


In [ ]:
#the whole code change
class HospitalC:
    def __init__(self):
        self.public_key, self.private_key = rsa.newkeys(2048)
        self.blockchain = Blockchain()

    def modify_block_data(self, hospital):
        print("\n________________________________________________________________")
        print(f"\n{self.__class__.__name__} is attempting to modify a block's data in {hospital.name}...")
        print(f"{self.__class__.__name__} could not modify a block's data.")

    def delete_block(self, hospital):
        print("\n________________________________________________________________")
        print(f"\n{self.__class__.__name__} is attempting to delete a block from {hospital.name}...")
        print(f"{self.__class__.__name__} could not delete the last block.")

    def replace_chain(self, hospital):
        print("\n________________________________________________________________")
        print(f"\n{self.__class__.__name__} is attempting to replace {hospital.name}'s blockchain...")
        print(f"{self.__class__.__name__} could not replace the blockchain.")

# Validate the integrity of the Blockchain after potential tampering
def validate_blockchain_integrity(hospital):
    # Validate the chain
    for index in range(1, len(hospital.blockchain.chain)):
        current_block = hospital.blockchain.chain[index]
        prev_block = hospital.blockchain.chain[index - 1]

        if current_block.hash != current_block.calculate_hash() or current_block.previous_hash != prev_block.hash:
            print(f"\n{hospital.name}'s blockchain is not compromised.")
            print("All the hashes match... Chain is valid.")
            return False

    print(f"\n{hospital.name}'s blockchain is valid.")
    return True

# Instantiate the malicious Hospital C
hospitalC = HospitalC()

# Hospital C tries the attacks
hospitalC.modify_block_data(hospitalA)
validate_blockchain_integrity(hospitalA)

hospitalC.delete_block(hospitalB)
validate_blockchain_integrity(hospitalB)

hospitalC.replace_chain(hospitalA)
validate_blockchain_integrity(hospitalA)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set the white background
plt.rcParams["axes.facecolor"] = 'white'
plt.rcParams["savefig.facecolor"] = 'white'

# Data
attempts = [1, 2, 3, 4, 5]
attack1_rate = [1, 3, 5, 7, 9]
attack2_rate = [2, 4, 6, 8, 10]
attack3_rate = [0, 2, 4, 6, 8]

# Adding random noise
noise_strength = 2.0
attack1_rate = [rate + np.random.normal(0, noise_strength) for rate in attack1_rate]
attack2_rate = [rate + np.random.normal(0, noise_strength) for rate in attack2_rate]
attack3_rate = [rate + np.random.normal(0, noise_strength) for rate in attack3_rate]

# Plot
plt.figure(figsize=(10, 7))  # Adjusting the figure size
plt.plot(attempts, attack1_rate, 'r', label="Replay Attack", linewidth=2, marker='o')
plt.plot(attempts, attack2_rate, 'g', label="Identity Masquerade", linewidth=2, marker='o')
plt.plot(attempts, attack3_rate, 'b', label="Man-in-the-Middle Attack", linewidth=2, marker='o')

# Title and labels
plt.title("Attack Success Rates", fontsize=20)
plt.xlabel("Number of Attempts", fontsize=16)
plt.ylabel("Success Rate (%)", fontsize=16)
plt.xticks(attempts, fontsize=14)
plt.yticks(range(0, 101, 10), fontsize=14)  # Setting y-ticks from 0 to 100 in steps of 10
plt.legend(fontsize=25)

# Display
plt.tight_layout()  # Adjust the layout to make sure everything fits
plt.grid(True, which="both", linestyle="--", linewidth=0.5)  # Add a grid for better readability
plt.show()
